## Simple LSTM for Sequence Classification 

IMDB sorunu için hızlı bir şekilde küçük bir LSTM geliştirebilir ve iyi bir doğruluk elde edebiliriz.

Bu model için gereken sınıfları ve işlevleri içe aktararak ve sonuçları kolayca yeniden oluşturabilmemizi sağlamak için rasgele sayı üretecini sabit bir değere başlatarak başlayalım.

In [2]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# fix random seed for reproducibility
numpy.random.seed(7)

IMDB veri setini yüklememiz gerekiyor. Veri kümesini ilk 5.000 kelimeyle sınırlandırıyoruz. Ayrıca veri kümesini train (%50) ve test (%50) kümelerine ayırdık.

In [3]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17465344/17464789 [==============================] - 31s 2us/step


Daha sonra, modelleme için hepsinin aynı uzunlukta olması için girdi dizilerini kesmemiz ve doldurmamız gerekiyor. Model, sıfır değerlerinin hiçbir bilgi taşımadığını öğrenecektir, bu nedenle diziler içerik açısından aynı uzunlukta değildir, ancak Keras'ta hesaplamayı gerçekleştirmek için aynı uzunlukta vektörler gereklidir.

In [4]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)


Artık LSTM modelimizi tanımlayabilir, compile ve fit edebiliriz

İlk katman, her bir sözcüğü temsil etmek için 32 uzunluk vektörü kullanan Embedded katmandır. Bir sonraki katman, 100 bellek birimine (akıllı nöronlar) sahip LSTM katmanıdır. Son olarak, bu bir sınıflandırma problemi olduğundan, problemdeki iki sınıf (iyi ve kötü) için 0 veya 1 tahminde bulunmak için tek bir nöron ve bir sigmoid aktivasyon fonksiyonu ile Dense bir çıktı katmanı kullanıyoruz.

In [5]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 771s 2s/step - loss: 0.6231 - accuracy: 0.6645 - val_loss: 0.3731 - val_accuracy: 0.8396
Epoch 2/3
391/391 [==============================] - 800s 2s/step - loss: 0.3296 - accuracy: 0.8664 - val_loss: 0.3268 - val_accuracy: 0.8660
Epoch 3/3
391/391 [==============================] - 806s 2s/step - loss: 0.2997 - accuracy

Uygun olduğunda, görünmeyen incelemelerde modelin performansını tahmin ederiz.

In [6]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.17%


In [2]:
# LSTM with Dropout for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
dropout (Dropout)            (None, 500, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 644s 2s/step - loss: 0.6015 - accuracy: 0.6334
Epoch 2/3
3